# Lab 8. Tensor Manipulation

In [10]:
import tensorflow as tf
import numpy as np
import pprint

tf.set_random_seed(777)  # for reproducibility

pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()
#sess.close()

## 1. Array and Slicing

In [2]:
# Array and slicing
t = np.array([[0., 1.], [2., 3.], [4., 5.], [6., 7.]])

pp.pprint(t)
print("t.ndim =", t.ndim) # rank
print("t.shape =", t.shape) # shape
print("t[0], t[1], t[-1] =", t[0], t[1], t[-1])
print("t[:3] =", t[:3])
print("t[1:-1] =", t[1:-1])

array([[0., 1.],
       [2., 3.],
       [4., 5.],
       [6., 7.]])
t.ndim = 2
t.shape = (4, 2)
t[0], t[1], t[-1] = [0. 1.] [2. 3.] [6. 7.]
t[:3] = [[0. 1.]
 [2. 3.]
 [4. 5.]]
t[1:-1] = [[2. 3.]
 [4. 5.]]


## 2. Shape, Rank, Axis
- Rank : [ 의 개수이다.
- Shape : [?1, ?2, ?3]로 나타내며, ?3은 가장 안쪽의 [ ] 안에 있는 엘리먼트의 개수 ?2는 [[ ]] 안에 있는 엘리먼트의 개수, ?3은 [[[ ]]]안에 있는 엘리먼트의 개수이다. ?의 개수는 rank의 수와 같다.
- Axis는 rank의 수와 같은만큼 존재하는데 가장 바깥쪽의 [가 axis 0이며 안쪽으로 들어갈수록 1씩 증가한다.
- 가장 안쪽의 axis는 -1로 표기하는데 가장 많이 사용한다.

In [45]:
# Rank(count of brackets) = 1, element = 4, Shape = [4]
t = tf.constant([1, 2, 3, 4])
tf.shape(t).eval()

array([4], dtype=int32)

In [54]:
# Rank = 2, Shape = [2, 4]
t = tf.constant([[1, 2, 3, 4],
                 [5, 6, 7, 8]])
tf.shape(t).eval()

array([2, 4], dtype=int32)

In [55]:
# Rank = 4, Shape = [1, 2, 3, 4]
t = tf.constant([[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
                  [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]]])
tf.shape(t).eval()

array([1, 2, 3, 4], dtype=int32)

In [57]:
[ # axis = 0 (1st brakets)
    [ # axis = 1 (2nd brackets)
        [ # axis = 2 (3th brackets)
            [1,2,3,4],  # axis = 3 or -1 (4th brackets)
            [5,6,7,8],
            [9,10,11,12]
        ],
        [
            [13,14,15,16],
            [17,18,19,20], 
            [21,22,23,24]
        ]
    ]
]

[[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
  [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]]]

## 3. Matmul vs Multiply
- [x,y][y,z]  = [x,z]
- 두 행렬의 shpae을 확인한다.
- 행렬의 곱셈이 일어나려면 y 값이 같이야 하며, 행렬의 곱셈이 완료되면 [x,z]의 shape이 된다.
- 행렬의 곱셈은 tf.matmul 함수를 사용한다.

In [59]:
matrix1 = tf.constant([[1., 2.], [3., 4.]])
matrix2 = tf.constant([[1.], [2.]])
print("Metrix 1 shape =", matrix1.shape)
print("Metrix 2 shape =", matrix2.shape)
tf.matmul(matrix1, matrix2).eval()

Metrix 1 shape = (2, 2)
Metrix 2 shape = (2, 1)


array([[ 5.],
       [11.]], dtype=float32)

In [61]:
# Warning: martix multiplication vs general multiplication
(matrix1 * matrix2).eval() # See below broadcasting

array([[1., 2.],
       [6., 8.]], dtype=float32)

## 4. Watch out broadcasting
- Shape이 맞지 않는 경우 수학적으로 연산이 안되는데, tensorflow에서는 broadcasting을 시켜 shape을 연산이 되도록 맞춰준다.

In [63]:
# Same shape
matrix1 = tf.constant([[3., 3.]]) # Rank = 2, Shape = [1, 2]
matrix2 = tf.constant([[2., 2.]]) # Rank = 2, Shape = [1, 2]
(matrix1 + matrix2).eval() # Rank = 2, Shape = [1, 2]

array([[5., 5.]], dtype=float32)

In [73]:
# Different shape case 1
matrix1 = tf.constant([[1., 2.]]) # Rank = 2, Shape = [1, 2]
matrix2 = tf.constant([3.]) # Rank = 1, Shape = [1]
# Broadcasting output = [[3., 3.]]
(matrix1 + matrix2).eval() # Rank = 2, Shape [1, 2]

array([[4., 5.]], dtype=float32)

In [72]:
# Different shape case 2
matrix1 = tf.constant([[1., 2.]]) # Rank = 2, Shape = [1, 2]
            # Broadcasting output = [1., 2.]
matrix2 = tf.constant([3., 4.]) # Rank = 1, Shape = [2]
            # Broadcasting output = [3., 4.]
(matrix1 + matrix2).eval() # Rank = 2, Shape = [1, 2]

array([[4., 6.]], dtype=float32)

In [71]:
# Different shape case 3
matrix1 = tf.constant([[1., 2.]]) # Rank = 2, Shape = [1, 2]
# Broadcasting output = [1., 2.], 
#                       [1., 2.]]
matrix2 = tf.constant([[3.], [4.]]) # Rank = 2, Shape = [2, 1]
# Broadcasting output = [[3., 3.], 
#                        [4., 4.]]
(matrix1 + matrix2).eval() # Rank = 2, Shape = 2

array([[4., 5.],
       [5., 6.]], dtype=float32)

## 5. Reduce Mean and Reduce Sum
- reduce_mean은 axis값에 따라 행렬의 평균을 계산하는 함수이다.
- redume_sum은 axis값에 따라 행렬의 합을 계산하는 함수이다.

In [74]:
# case 1
tf.reduce_mean([1, 2], axis = 0).eval()

1

In [75]:
# case 2
tf.reduce_mean([1., 2.], axis = 0).eval()

1.5

In [81]:
# case 3
x = [[1., 2.],
     [3., 4.]]
tf.reduce_mean(x, axis = 0).eval() # (1+3)/2, (2+4)/2

array([2., 3.], dtype=float32)

In [85]:
# case 4
x = [[1., 2.],
     [3., 4.]]
tf.reduce_mean(x, axis = 1).eval() # (1+2)/2, (3+4)/2

array([1.5, 3.5], dtype=float32)

In [83]:
# case 5
x = [[1., 2.],
     [3., 4.]]
tf.reduce_mean(x, axis = -1).eval() # same case 4 (max axis)

array([1.5, 3.5], dtype=float32)

In [84]:
# case 6
x = [[1., 2.],
     [3., 4.]]
tf.reduce_mean(x).eval() # ( ((1+2)/2) + (3+4)/2 ) / 2

2.5

In [86]:
# case 7
x = [[1., 2.],
     [3., 4.]]
tf.reduce_sum(x, axis = 0).eval() # (1+3), (2+4)

array([4., 6.], dtype=float32)

In [87]:
# case 8
x = [[1., 2.],
     [3., 4.]]
tf.reduce_sum(x, axis = -1).eval() # (1+2), (3+4)

array([3., 7.], dtype=float32)

In [88]:
# case 9
x = [[1., 2.],
     [3., 4.]]
tf.reduce_sum(x).eval() # (1+2) + (3 + 4)

10.0

In [93]:
# case 10
x = [[1., 2.],
     [3., 4.]]
tf.reduce_mean(tf.reduce_sum(x, axis = -1)).eval() # case 8 / 2

5.0

## 6. Argmax
- axis에 따라 행렬에서 가장 큰수에 해당하는 index 값을 찾아주는 함수이다. 

In [14]:
# case 1
x = [[0, 1, 2],
     [2, 1, 0]]
tf.argmax(x, axis = 0).eval() # return array index, not return value

array([1, 0, 0])

In [12]:
# case 2
x = [[0, 1, 2],
     [2, 1, 0]]
tf.argmax(x, axis = 1).eval()

array([2, 0])

In [98]:
# case 3
x = [[0, 1, 2],
     [2, 1, 0]]
tf.argmax(x, axis = -1).eval() # same axis 1

array([2, 0])

## 7. Reshape
- 일반적으로 machine learning에서 traning data는 수정을 하지 않으므로, 가장 안쪽의 엘리먼트는 수정을 하지 않는것이 일반적이다.

In [100]:
# Origin
t = np.array([[[0, 1, 2],
               [3, 4, 5]],
              [[6, 7, 8],
               [9, 10, 11]]])
t.shape

(2, 2, 3)

In [102]:
# Reshape 1
tf.reshape(t, shape=[-1, 3]).eval() # Do not reshape last element

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [108]:
# Reshape 2
tf.reshape(t, shape=[-1, 1, 3]).eval()

array([[[ 0,  1,  2]],

       [[ 3,  4,  5]],

       [[ 6,  7,  8]],

       [[ 9, 10, 11]]])

In [107]:
# Reshape 3
tf.reshape(t, shape=[-1, 2, 3]).eval() # same shape[2, 2, 3]

array([[[ 0,  1,  2],
        [ 3,  4,  5]],

       [[ 6,  7,  8],
        [ 9, 10, 11]]])

## 8. Reshape (Squeeze and Expand)
- squeeze는 다차원의 행렬을 1차원으로 변경해준다.
- expand_dims는 주어진 행렬의 차원을 다차원의 행렬로 변경해준다.

In [21]:
# Squeeze
tf.squeeze([[0], [1], [2]]).eval()

array([0, 1, 2], dtype=int32)

In [117]:
# Expand 1
tf.expand_dims([0, 1, 2], 0).eval()

array([[0, 1, 2]], dtype=int32)

In [119]:
# Expand 2
tf.expand_dims([0, 1, 2], 1).eval()

array([[0],
       [1],
       [2]], dtype=int32)

## 9. One hot
- one_hot은 각 엘리먼트의 수에 해당하는 자리만 hot하게 1로 만들어 준다. depth는 엘리먼트의 종류의 수이다. 그래야 자리로 구분할수 있기 때문이다.
- case 1에서 엘리먼트는 0, 1, 2, 3 즉 4종류의 수로 이루어진 행렬이므로 depth는 4가 되며, 0에 해당하는 byte masking은 [[1.0, 0, 0, 0]]이 된다.
- one_hot 함수의 결과는 항상 1차원이 늘어나기 때문에 필요시 reshape을 사용해야 한다.

In [29]:
# case 1
tf.one_hot([[0], [1], [2], [3]], depth=4).eval()

array([[[1., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       [[0., 0., 1., 0.]],

       [[0., 0., 0., 1.]]], dtype=float32)

In [25]:
# case 2
t = tf.one_hot([[0], [3], [1], [2]], depth=4).eval()
tf.reshape(t, shape=[-1, 4]).eval()

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

## 10. Casting

In [128]:
# case 1
tf.cast([1.1, 2.2, 3.3, 4.9], tf.int32).eval()

array([1, 2, 3, 4], dtype=int32)

In [129]:
# case 2
tf.cast([True, False, 1 == 1, 0 == 1], tf.int32).eval()

array([1, 0, 1, 0], dtype=int32)

## 11. Stack
- axis에 따라 행렬을 쌓을수 있게 해주는 함수이다.

In [130]:
# case 1
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
tf.stack([x, y, z]).eval()

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)

In [131]:
# case 2
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
tf.stack([x, y, z], axis = 0).eval()

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)

In [133]:
# case 3
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
tf.stack([x, y, z], axis = 1).eval()

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]], dtype=int32)

In [134]:
# case 3
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
tf.stack([x, y, z], axis = -1).eval()

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]], dtype=int32)

## 12. Ones and Zeros like
- 주어진 행렬(tensor)의 모든 엘리먼트를 0또는 1로 채울수 있다.

In [135]:
# case 1
x = [[0, 1, 2],
     [2, 1, 0]]
tf.ones_like(x).eval()

array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)

In [136]:
# case 2
x = [[0, 1, 2],
     [2, 1, 0]]
tf.zeros_like(x).eval()

array([[0, 0, 0],
       [0, 0, 0]], dtype=int32)

## 13. Zip
- 두대 이상의 행렬에서 엘리먼트를 순서에 대로 묶에서 리턴해줄때 사용하는 함수이다.
- 일반적으로 for loop에서 많이 사용한다.

<img src="http://blog.londasfiles.com/wp-content/uploads/2017/10/zipper.jpg" width="200" />

In [143]:
# case 1
for x, y in zip([1, 2, 3], [4, 5, 6]):
    print("x =", x, "\ty =", y)

x = 1 	y = 4
x = 2 	y = 5
x = 3 	y = 6


In [144]:
# case 2
for x, y, z in zip([1, 2, 3], [4, 5, 6], [7, 8, 9]):
    print("x =", x, "\ty =", y, "\tz =", z)

x = 1 	y = 4 	z = 7
x = 2 	y = 5 	z = 8
x = 3 	y = 6 	z = 9
